In [1]:
!pip install tensorflow opencv-python pillow scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/fruits.zip /content/
!unzip -q /content/fruits.zip -d /content/dataset
!ls /content/dataset

In [ ]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Paths
dataset_dir = '/content/dataset/Fruit And Vegetable Diseases Dataset'
output_dir = '/content/output_dataset'

# Create output directories
os.makedirs(output_dir, exist_ok=True)
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Process each class
classes = os.listdir(dataset_dir)
for cls in classes:
    class_dir = os.path.join(dataset_dir, cls)
    images = [img for img in os.listdir(class_dir) if img.endswith(('.jpg', '.png', '.jpeg'))]

    # Optional: limit number of images per class
    images = images[:300]

    train_val, test = train_test_split(images, test_size=0.1, random_state=42)
    train, val = train_test_split(train_val, test_size=0.2222, random_state=42)

    for split, split_images in zip(['train', 'val', 'test'], [train, val, test]):
        split_class_dir = os.path.join(output_dir, split, cls)
        os.makedirs(split_class_dir, exist_ok=True)
        for img_name in split_images:
            src = os.path.join(class_dir, img_name)
            dst = os.path.join(split_class_dir, img_name)
            shutil.copy(src, dst)

print("✅ Dataset successfully split into train / val / test.")


In [ ]:
import os
import json
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Constants
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30

# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    '/content/output_dataset/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/output_dataset/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Save class labels
with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)

# Build the model
base_model = MobileNetV2(include_top=False, input_shape=(150, 150, 3), weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True)

# Train Phase 1
model.fit(train_generator, epochs=EPOCHS, validation_data=val_generator, callbacks=[early_stop, checkpoint])

# Fine-tune last few layers
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Train Phase 2 (fine-tuning)
model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[early_stop])

# Save final model
model.save("healthy_vs_rotten.h5")
print("✅ Model training complete and saved as healthy_vs_rotten.h5")


In [ ]:
from google.colab import files
files.download('healthy_vs_rotten.h5')